In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/09/12 01:38:16 WARN Utils: Your hostname, dash_surface resolves to a loopback address: 127.0.1.1; using 172.17.62.49 instead (on interface eth0)
22/09/12 01:38:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 01:38:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F

### Load consumer data

In [4]:
sdf_consumer = spark.read.csv('../data/tables/tbl_consumer.csv', sep='|', header=True)
sdf_consumer.show(5, truncate=False)

+-----------------+-----------------------------+-----+--------+------+-----------+
|name             |address                      |state|postcode|gender|consumer_id|
+-----------------+-----------------------------+-----+--------+------+-----------+
|Yolanda Williams |413 Haney Gardens Apt. 742   |WA   |6935    |Female|1195503    |
|Mary Smith       |3764 Amber Oval              |NSW  |2782    |Female|179208     |
|Jill Jones MD    |40693 Henry Greens           |NT   |862     |Female|1194530    |
|Lindsay Jimenez  |00653 Davenport Crossroad    |NSW  |2780    |Female|154128     |
|Rebecca Blanchard|9271 Michael Manors Suite 651|WA   |6355    |Female|712975     |
+-----------------+-----------------------------+-----+--------+------+-----------+
only showing top 5 rows



In [5]:
sdf_consumer.count()

499999

### Load transactional data

In [5]:
sdf_transactions = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot')
sdf_transactions.show(10, truncate=False)

+-------+------------+------------------+------------------------------------+--------------+
|user_id|merchant_abn|dollar_value      |order_id                            |order_datetime|
+-------+------------+------------------+------------------------------------+--------------+
|18478  |62191208634 |63.255848959735246|949a63c8-29f7-4ab0-ada4-99ac50a88952|2021-08-20    |
|2      |15549624934 |130.3505283105634 |6a84c3cf-612a-4574-835b-144a47353eff|2021-08-20    |
|18479  |64403598239 |120.15860593212783|b10dcc33-e53f-4254-863c-de5266810cbc|2021-08-20    |
|3      |60956456424 |136.6785200286976 |0f09c5a5-784e-4477-b049-8ee4dd069b7b|2021-08-20    |
|18479  |94493496784 |72.96316578355305 |f6c78c1a-4600-4c5f-8e97-6e9eb534b586|2021-08-20    |
|3      |76819856970 |448.529684285612  |5ace6a24-cdf0-4aa3-b571-1d9406b352b5|2021-08-20    |
|18479  |67609108741 |86.4040605836911  |d0e180f0-cb06-42a3-bd1a-c47dca15bc55|2021-08-20    |
|3      |34096466752 |301.5793450525113 |6fb1ff48-24bb-4f97-

In [8]:
sdf_transactions.count()

3643266

### Read consumer details (joining table)

In [6]:
sdf_userdetails = spark.read.parquet('../data/tables/consumer_user_details.parquet')
sdf_userdetails.show(10, truncate=False)

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|1      |1195503    |
|2      |179208     |
|3      |1194530    |
|4      |154128     |
|5      |712975     |
|6      |407340     |
|7      |511685     |
|8      |448088     |
|9      |650435     |
|10     |1058499    |
+-------+-----------+
only showing top 10 rows



In [10]:
sdf_userdetails.count()

499999

### Join transactional data with consumer data

In [7]:
sdf_usertransaction = sdf_userdetails.join(sdf_transactions, on='user_id')
sdf_consumer_transaction = sdf_usertransaction.join(sdf_consumer, on='consumer_id')
sdf_consumer_transaction.show(5)

+-----------+-------+------------+------------------+--------------------+--------------+---------+--------------------+-----+--------+------+
|consumer_id|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|     name|             address|state|postcode|gender|
+-----------+-------+------------+------------------+--------------------+--------------+---------+--------------------+-----+--------+------+
|        551|   3471| 68216911708|  39.2325218595659|c921263e-489e-45a...|    2021-08-21|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 77698107389| 90.16967584866273|aaea14f5-48e8-4ef...|    2021-08-21|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 46451548968|3.8405569898888428|7e8e7fca-04a1-4f2...|    2021-08-19|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 35556933338| 66.18325079406301|13b0be6a-20b7-439...|    2021-08-22|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|

In [8]:
# Only keeping useful features
sdf_consumer_transaction = sdf_consumer_transaction.select("consumer_id","user_id",'merchant_abn','dollar_value',
                            'order_datetime','state','postcode','gender')
sdf_consumer_transaction.show(5)                   

+-----------+-------+------------+------------------+--------------+-----+--------+------+
|consumer_id|user_id|merchant_abn|      dollar_value|order_datetime|state|postcode|gender|
+-----------+-------+------------+------------------+--------------+-----+--------+------+
|        551|   3471| 68216911708|  39.2325218595659|    2021-08-21|  NSW|    2021|  Male|
|        551|   3471| 77698107389| 90.16967584866273|    2021-08-21|  NSW|    2021|  Male|
|        551|   3471| 46451548968|3.8405569898888428|    2021-08-19|  NSW|    2021|  Male|
|        551|   3471| 35556933338| 66.18325079406301|    2021-08-22|  NSW|    2021|  Male|
|        551|   3471| 91772719127|33.692747732339996|    2021-08-14|  NSW|    2021|  Male|
+-----------+-------+------------+------------------+--------------+-----+--------+------+
only showing top 5 rows



In [13]:
# Checking final count to see if it joined properly
sdf_consumer_transaction.count()

3643266

In [14]:
# Looking at descriptive statistics for outlier analysis
sdf_consumer_transaction.describe()

summary,consumer_id,user_id,merchant_abn,dollar_value,state,postcode,gender
count,3643266,3643266,3643266,3643266,3643266,3643266,3643266
mean,753342.8796423319,12038.386185362255,5.542282307467807E10,166.27987538249533,null,4043.809383119432,null
stddev,435447.4489588989,6951.397455767658,2.532956222776528...,520.5869486446928,null,1780.2337681551908,null
min,30,1,10023283211,9.756658099412162E-8,ACT,1001,Female
max,1499867,24081,99990536339,105193.88578925544,WA,9999,Undisclosed


In [15]:
sdf_consumer_transaction.printSchema()

root
 |-- consumer_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)



### Deal with gender

In [16]:
# Checking gender categories
sdf_consumer_transaction.groupBy('gender').count()

gender,count
Undisclosed,369963
Female,1619059
Male,1654244


With our genders, it may be a promising feature which woucl dbe used further along the line. Thus, we want to keep it for future references. However, there are people who wish to not disclose their gender and a gender feature as 'undisclosed' has appeared. The following code deals with attempting to find if there has been any past that customers who have said 'undisclosed' have provided us.

In [24]:
# Find all undisclosed information user id, then turn it into a  list

undisc_user_id = sdf_consumer_transaction.select(F.col("user_id")).where(F.col('gender') == "Undisclosed")
undisc_user_id = undisc_user_id.distinct()

In [25]:
undisc_user_id = undisc_user_id.toPandas()["user_id"].values.tolist()

In [21]:
# Make sdf of all labelled genders

disc_user_id = sdf_consumer_transaction.where(F.col('gender') != "Undisclosed")
disc_user_id = disc_user_id.distinct()

In [23]:
disc_user_id = disc_user_id.toPandas()["user_id"].values.tolist()

In [26]:
# Observe if there's any customers who have previously put in a gender

for undisc in undisc_user_id:
    if undisc in disc_user_id:
        print(undisc)

For the first set of data, doesn't seem like there's any overlap. Smoge useless analysis

In [21]:
# Only keep male and female genders, make boolean
# sdf_consumer_transaction = sdf_consumer_transaction.where(
#     (F.col('gender') == 'Male') | (F.col('gender') == 'Female')
# )
# sdf_consumer_transaction = sdf_consumer_transaction.withColumn(
#     'gender',
#     (F.col('gender') == 'Male').cast('BOOLEAN')
# )
# sdf_consumer_transaction.count()


3273303

In [17]:
# Checking if there is only correct states
sdf_consumer_transaction.groupBy('state').count()

state,count
NT,51779
ACT,33596
SA,414261
TAS,134807
WA,577024
QLD,538942
VIC,842008
NSW,1050849


### Round dollar values to 2dp

In [9]:
# Round to 2 decimal places, and remove transactions equal to zero
# Discuss min value
min_value = 5
sdf_consumer_transaction = sdf_consumer_transaction.withColumn('dollar_value', F.round(F.col('dollar_value'), 2))
sdf_consumer_transaction = sdf_consumer_transaction.where(
    (F.col('dollar_value') > min_value)
)

### Check ABN validity

In [17]:
# Make sure ABN is valid, takes in long

def validateABN(merchant_abn):

    str_abn = str(merchant_abn)
    i = 0
    sum = 0

    for digit in str_abn:

        digit = int(digit)
        
        if i == 0:
            digit -= 1
            sum += digit * 10
        elif i == 1:
            sum += digit
        elif i == 2:
            sum += digit * 3
        elif i == 3:
            sum += digit * 5
        elif i == 4:
            sum += digit * 7
        elif i == 5:
            sum += digit * 9
        elif i == 6:
            sum += digit * 11
        elif i == 7:
            sum += digit * 13
        elif i == 8:
            sum += digit * 15
        elif i == 9:
            sum += digit * 17
        elif i == 10:
            sum += digit * 19

        i += 1
    if sum % 89 == 0:
        return True
    else:
        return False

In [11]:
# Create a list of all row values, used for validating ABN

sdf_list = sdf_consumer_transaction.select("merchant_abn").collect()

In [12]:
sdf_len = len(sdf_list)
sdf_len

3479953

In [18]:
# Find any merchants without a valid ABN

i = 0
invalidABN = []

while i < sdf_len:
    abn = str(sdf_list[i].__getitem__('merchant_abn'))
    if validateABN(abn) == False:
        invalidABN.append(abn)
    i += 1

In [24]:
len(invalidABN)

3442696

In [23]:
invalidABN

['62191208634',
 '15549624934',
 '64403598239',
 '60956456424',
 '94493496784',
 '76819856970',
 '67609108741',
 '34096466752',
 '70501974849',
 '49891706470',
 '46804135891',
 '33064796871',
 '44160392990',
 '68435002949',
 '70033549200',
 '41944909975',
 '41705715409',
 '29566626791',
 '32361057556',
 '47663262928',
 '30458474292',
 '82065156333',
 '16096277862',
 '21025433654',
 '76819856970',
 '96161947306',
 '57900494384',
 '32234779638',
 '68216911708',
 '75034515922',
 '11237511112',
 '23661821077',
 '13565102775',
 '70620117107',
 '67202032418',
 '86578477987',
 '71787414647',
 '86578477987',
 '64203420245',
 '96680767841',
 '15299889494',
 '33651513345',
 '32461318592',
 '63290521567',
 '19249968599',
 '56559128138',
 '72472909171',
 '58476363584',
 '32234779638',
 '43186523025',
 '63290521567',
 '34179569263',
 '23633724513',
 '45380641195',
 '77679081913',
 '89640578182',
 '49505931725',
 '45337626814',
 '87921002735',
 '39869100896',
 '28341499489',
 '12173100693',
 '213591

### Check for valid postcodes in Australia

In [23]:
# Make sure post codes in right range and only digits
min_postcode = '0200'
max_postcode = '9999'

# Check if only digits
# sdf_consumer_transaction = sdf_consumer_transaction.where(F.col("postcode").cast('string'))

sdf_consumer_transaction = sdf_consumer_transaction.where(
    (F.col('postcode') >= min_postcode)
    & (F.col('postcode') <= max_postcode)
)


In [39]:
# Checking descriptive statistics again
sdf_consumer_transaction.describe()

summary,consumer_id,user_id,merchant_abn,dollar_value,state,postcode,gender
count,3479953,3479953,3479953,3479953,3479953,3479953,3479953
mean,753335.9036369169,12038.683883661648,5.545921570646908E10,173.960070414744,null,4043.795159589799,null
stddev,435468.2355945888,6951.87581999312,2.535943687229347...,531.4256728713924,null,1780.3214030078946,null
min,30,1,10023283211,5.01,ACT,1001,Female
max,1499867,24081,99990536339,105193.89,WA,9999,Undisclosed


In [ ]:
# Deal with NULL values

In [ ]:
# Export cleaned data